# 🧠 Text-to-SQL Results Analysis Notebook

In this notebook, we summarize and analyze the results of various experiments run using the `text2sql-eval-toolkit`.

In [4]:
# Auto-reload for dev
%load_ext autoreload
%autoreload 2

from pathlib import Path
from text2sql_eval_toolkit.analysis.report_tools import collect_results

output_folder = Path().resolve().parent / "data" / "results"
results = collect_results(output_folder)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
all_benchmarks = list(results[0].keys())
all_benchmarks

['bird_mini_dev_sqlite',
 'bird_mini_dev_postgres',
 'beaver',
 'archer_en_dev',
 'spider_dev',
 'spider_realistic']

In [11]:
benchmark_id = "bird_mini_dev_sqlite"
all_pipelines = list(results[0][benchmark_id][2].keys())
all_pipelines

['wxai:meta-llama/llama-3-3-70b-instruct-greedy-zero-shot-chatapi',
 'wxai:ibm/granite-4-h-small-greedy-zero-shot-chatapi',
 'wxai:meta-llama/llama-4-maverick-17b-128e-instruct-fp8-greedy-zero-shot-chatapi',
 'wxai:openai/gpt-oss-120b-greedy-zero-shot-chatapi',
 'wxai:openai/gpt-oss-120b-agentic-baseline0-3attempts',
 'wxai:openai/gpt-oss-120b-agentic-baseline1-3attempts',
 'wxai:openai/gpt-oss-120b-agentic-baseline2-3attempts',
 'wxai:openai/gpt-oss-120b-agentic-baseline3-3attempts',
 'wxai:openai/gpt-oss-120b-agentic-baseline4-3attempts',
 'wxai:openai/gpt-oss-120b-agentic-baseline5-3attempts']

# Comparing Pipelines

In [12]:
pipeline_ids_to_compare = [
    'wxai:ibm/granite-4-h-small-greedy-zero-shot-chatapi',
    'wxai:meta-llama/llama-4-maverick-17b-128e-instruct-fp8-greedy-zero-shot-chatapi',
    'wxai:openai/gpt-oss-120b-greedy-zero-shot-chatapi',
]

In [13]:
pipeline_id = pipeline_ids_to_compare[2]
results[0][benchmark_id][2][pipeline_id]

{'execution_accuracy': {'average': 0.51, 'stddev': 0.5003574393264106},
 'non_empty_execution_accuracy': {'average': 0.51,
  'stddev': 0.5003574393264106},
 'subset_non_empty_execution_accuracy': {'average': 0.568,
  'stddev': 0.4955336952660278},
 'logic_execution_accuracy': {'average': 0.586, 'stddev': 0.4926266187605602},
 'bird_execution_accuracy': {'average': 0.532, 'stddev': 0.49933492732594276},
 'is_sqlglot_parsable': {'average': 0.99, 'stddev': 0.07745872803616538},
 'is_sqlparse_parsable': {'average': 0.996, 'stddev': 0.0},
 'sqlglot_equivalence': {'average': 0.046, 'stddev': 0.2100960353563744},
 'sqlglot_optimized_equivalence': {'average': 0.034,
  'stddev': 0.18176243541304832},
 'sqlparse_equivalence': {'average': 0.0, 'stddev': 0.0},
 'sql_exact_match': {'average': 0.032, 'stddev': 0.17651866924992204},
 'sql_syntactic_equivalence': {'average': 0.048, 'stddev': 0.2143887173353016},
 'eval_error': {'average': 0.0, 'stddev': 0.0},
 'df_error': {'average': 0.01, 'stddev': 0

# Comparing Metrics

In [14]:
import json
from text2sql_eval_toolkit.utils import (
    get_benchmark_info,
    get_default_eval_filename,
    Path,
)


def get_examples(data, pipeline_key, conditions, k=None):
    """
    Filters records from the data array based on specified conditions under a given pipeline key.

    Parameters:
    - data: List of dictionaries, each containing a 'predictions' field.
    - pipeline_key: The key under 'predictions' to look for evaluation metrics.
    - conditions: Dictionary of conditions to match, e.g., {'execution_accuracy': 0, 'subset_non_empty_execution_accuracy': 1}
    - k: Maximum number of matching records to return.

    Returns:
    - List of up to k records that match all specified conditions.
    """
    matching_records = []

    for record in data:
        try:
            evaluation = record["predictions"][pipeline_key]["evaluation"]
            if all(
                evaluation.get(metric) == value for metric, value in conditions.items()
            ):
                matching_records.append(record)
                if k is not None and len(matching_records) >= k:
                    break
        except KeyError:
            continue  # Skip records that don't have the expected structure

    return matching_records


In [15]:
import pandas as pd
from IPython.display import display, Markdown, clear_output
from text2sql_eval_toolkit.utils import parse_dataframe, truncate_dataframe, get_gt_sqls
from text2sql_eval_toolkit.analysis.error_analysis import safe_snippet


def show_example(record: dict, pipeline_id: str):
    pred = record["predictions"][pipeline_id]
    gt_sqls = get_gt_sqls(record)

    gt_dfs = []
    raw_gt_dfs = record.get("gt_df", [])
    if isinstance(raw_gt_dfs, str):
        gt_dfs = [parse_dataframe(raw_gt_dfs)]
    else:
        for df in record.get("gt_df", []):
            try:
                gt_dfs.append(parse_dataframe(df))
            except Exception as e:
                gt_dfs.append(f"⚠️ Error loading GT DF: {e}")

    pred_df = None
    pred_df_error = None
    if "predicted_df" in pred:
        try:
            pred_df = parse_dataframe(pred["predicted_df"])
        except Exception as e:
            pred_df_error = f"⚠️ Error loading predicted_df: {e}"

    clear_output(wait=True)
    utterance = (
        record["page_content"]
        if "page_content" in record
        else record["question"]
        if "question" in record
        else record["utterance"]
    )
    question_id = record["id"] if "id" in record else record["_id"]
    display(Markdown(f"### ❓ Question ID: {question_id}\nQuestion: {utterance}"))
    display(Markdown("### Ground Truth SQL(s)"))
    for sql in gt_sqls:
        display(Markdown(f"```sql\n{sql}\n```"))

    display(Markdown("### Predicted SQL"))
    display(Markdown(f"```sql\n{pred['predicted_sql']}\n```"))

    display(Markdown("### 📊 Evaluation Metrics"))
    display(pd.DataFrame([pred["evaluation"]]))

    display(Markdown("### 📘 Ground Truth Result(s)"))
    for i, df in enumerate(gt_dfs):
        display(Markdown(f"**Result {i + 1}:**"))
        if isinstance(df, pd.DataFrame):
            display(truncate_dataframe(df))
        else:
            display(Markdown(df))

    display(Markdown("### 📕 Predicted Result"))
    if pred_df is not None:
        display(truncate_dataframe(pred_df))
    elif pred_df_error:
        display(Markdown(pred_df_error))

    display(Markdown(f"### 🧠 Prompt\n```\n{safe_snippet(pred['prompt'])}...\n```"))

In [22]:
benchmark_id = "spider_dev"
benchmark_info = get_benchmark_info(benchmark_id)
predictions_path = benchmark_info["predictions_path"]
eval_path = Path(get_default_eval_filename(predictions_path))
print(f"Evaluation file path: {eval_path}")
pred_eval_data = json.load(eval_path.open("r"))

pipeline_id = "wxai:openai/gpt-oss-120b-greedy-zero-shot-chatapi"
conditions = {
    "execution_accuracy": 0,
    "subset_non_empty_execution_accuracy": 0,
    "llm_score": 1,
}

examples = get_examples(
    pred_eval_data,
    pipeline_id,
    conditions,
)
print(f"{len(examples)} samples found meeting the condition: {conditions}")

Evaluation file path: /Users/oktie/code/text2sql/text2sql-eval-toolkit/data/results/spider_dev-predictions_eval.json
172 samples found meeting the condition: {'execution_accuracy': 0, 'subset_non_empty_execution_accuracy': 0, 'llm_score': 1}


In [23]:
show_example(examples[1], pipeline_id)

### ❓ Question ID: 22
Question: Show the stadium name and the number of concerts in each stadium.

### Ground Truth SQL(s)

```sql
SELECT T2.name ,  count(*) FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id GROUP BY T1.stadium_id
```

### Predicted SQL

```sql
SELECT
    s.Name AS stadium_name,
    COUNT(c.concert_ID) AS concert_count
FROM stadium s
LEFT JOIN concert c
    ON c.Stadium_ID = s.Stadium_ID
GROUP BY s.Stadium_ID, s.Name
```

...

**Result 1:**

,Name,count(*)
0,Stark's Park,1
1,Glebe Park,1
2,Somerset Park,2
3,Recreation Park,1
4,Balmoor,1


### 📕 Predicted Result

,stadium_name,concert_count
0,Stark's Park,1
1,Somerset Park,2
2,Bayview Stadium,0
3,Hampden Park,0
4,Forthbank Stadium,0
5,Gayfield Park,0
6,Recreation Park,1
7,Balmoor,1
8,Glebe Park,1


### 🧠 Prompt
```
Your task is to convert a natural language question into an accurate SQL query using the given sqlite database schema.

**Question:**:
Show the stadium name and the number of concerts in each stadium.

**Database Engine / Dialect:**:
sqlite

**Schema:**
Table: stadium
  Columns:
    - Stadium_ID (INT) (Primary Key) # Example values: 1, 2, 3, 4, 5
    - Location (TEXT) # Example values: Raith Rovers, Ayr United, East Fife, Queen's Park, Stirling Albion
    - Name (TEXT) # Example values: Stark's Park, Somerset Park, Bayview Stadium, Hampden Park, Forthbank Stadium
    - Capacity (INT) # Example values: 10104, 11998, 2000, 52500, 3808
    - Highest (INT) # Example values: 4812, 2363, 1980, 1763, 1125
    - Lowest (INT) # Example values: 1294, 1057, 533, 466, 404
    - Average (INT) # Example values: 2106, 1477, 864, 730, 642

Table: singer
  Columns:
    - Singer_ID (INT) (Primary Key) # Example values: 1, 2, 3, 4, 5
    - Name (TEXT) # Example values: Joe Sharp, Timbaland, Justin Brown, Rose White, John Nizinik
    - Country (TEXT) # Example values: Netherlands, United States, France
    - Song_Name (TEXT) # Example values: You, Dangerous, Hey Oh, Sun, Gentleman
    - Song_release_year (TEXT) # Example values: 1992, 2008, 2013, 2003, 2014
    - Age (INT) # Example values: 52, 32, 29, 41, 43
    - Is_male (BOOL) # Example values: F, T

Table: concert
  Columns:
    - concert_ID (INT) (Primary Key) # Example values: 1, 2, 3, 4, 5
    - concert_Name (TEXT) # Example values: Auditions, Super bootcamp, Home Visits, Week 1, Week 2
    - Theme (TEXT) # Example values: Free choice, Free choice 2, Bleeding Love, Wide Awake, Happy Tonight
    - Stadium_ID (TEXT) # Example values: 1, 2, 10, 9, 7
    - Year (TEXT) # Example values: 2014, 2015

Table: singer_in_concert
  Columns:
    - concert_ID (INT) (Primary Key) # Example values: 1, 2, 3, 4, 5
    - Singer_ID (TEXT) (Primary Key) # Example values: 2, 3, 5, 6, 4


**Instructions:**
- Only use columns listed in the schema.
- Do not use any other columns or tables not mentioned in the schema.
- Ensure the SQL query is valid and executable.
- Use proper SQL syntax and conventions.
- Generate a complete SQL query that answers the question.
- Use the correct SQL dialect for the database, i.e., sqlite.
- Do not include any explanations or comments in the SQL output.
- Your output must start with ```sql and end with ```.

Question: Show the stadium name and the number of concerts in each stadium....
```

In [25]:
benchmark_id = "bird_mini_dev_sqlite"
benchmark_info = get_benchmark_info(benchmark_id)
predictions_path = benchmark_info["predictions_path"]
eval_path = Path(get_default_eval_filename(predictions_path))
print(f"Evaluation file path: {eval_path}")
pred_eval_data = json.load(eval_path.open("r"))

pipeline_id = "wxai:openai/gpt-oss-120b-greedy-zero-shot-chatapi"
conditions = {
    "execution_accuracy": 0,
    "subset_non_empty_execution_accuracy": 1,
    # "llm_score": 1,
}

examples = get_examples(
    pred_eval_data,
    pipeline_id,
    conditions,
)
print(f"{len(examples)} samples found meeting the condition: {conditions}")

Evaluation file path: /Users/oktie/code/text2sql/text2sql-eval-toolkit/data/results/bird_mini_dev_sqlite-predictions_eval.json
29 samples found meeting the condition: {'execution_accuracy': 0, 'subset_non_empty_execution_accuracy': 1}


In [26]:
show_example(examples[0], pipeline_id)

### ❓ Question ID: 1486
Question: Is it true that more SMEs pay in Czech koruna than in euros? If so, how many more?

### Ground Truth SQL(s)

```sql
SELECT SUM(Currency = 'CZK') - SUM(Currency = 'EUR') FROM customers WHERE Segment = 'SME'
```

### Predicted SQL

```sql
SELECT
    SUM(CASE WHEN Segment = 'SME' AND Currency = 'CZK' THEN 1 ELSE 0 END) AS czk_count,
    SUM(CASE WHEN Segment = 'SME' AND Currency = 'EUR' THEN 1 ELSE 0 END) AS eur_count,
    SUM(CASE WHEN Segment = 'SME' AND Currency = 'CZK' THEN 1 ELSE 0 END) -
    SUM(CASE WHEN Segment = 'SME' AND Currency = 'EUR' THEN 1 ELSE 0 END) AS difference
FROM customers
```

...

**Result 1:**

,SUM(Currency = 'CZK') - SUM(Currency = 'EUR')
0,23505


### 📕 Predicted Result

,czk_count,eur_count,difference
0,25134,1629,23505


### 🧠 Prompt
```
Your task is to convert a natural language question into an accurate SQL query using the given sqlite database schema.

**Question:**:
Is it true that more SMEs pay in Czech koruna than in euros? If so, how many more?

**Database Engine / Dialect:**:
sqlite

**Schema:**
Table: customers
  Columns:
    - CustomerID (INTEGER) (Primary Key) # Example values: 3, 5, 6, 7, 9
    - Segment (TEXT) # Example values: SME, LAM, KAM
    - Currency (TEXT) # Example values: EUR, CZK

Table: gasstations
  Columns:
    - GasStationID (INTEGER) (Primary Key) # Example values: 44, 45, 46, 47, 48
    - ChainID (INTEGER) # Example values: 13, 6, 23, 33, 4
    - Country (TEXT) # Example values: CZE, SVK
    - Segment (TEXT) # Example values: Value for money, Premium, Other, Noname, Discount

Table: products
  Columns:
    - ProductID (INTEGER) (Primary Key) # Example values: 1, 2, 3, 4, 5
    - Description (TEXT) # Example values: Rucní zadání, Nafta, Special, Super, Natural

Table: transactions_1k
  Columns:
    - TransactionID (INTEGER) (Primary Key) # Example values: 1, 2, 3, 4, 5
    - Date (DATE) # Example values: 2012-08-24, 2012-08-23, 2012-08-25, 2012-08-26
    - Time (TEXT) # Example values: 09:41:00, 10:03:00, 13:53:00, 08:49:00, 08:53:00
    - CustomerID (INTEGER) # Example values: 31543, 46707, 7654, 17373, 7881
    - CardID (INTEGER) # Example values: 486621, 550134, 684220, 536109, 99745
    - GasStationID (INTEGER) # Example values: 3704, 656, 741, 1152, 636
    - ProductID (INTEGER) # Example values: 2, 23, 5, 11, 7
    - Amount (INTEGER) # Example values: 28, 18, 1, 5, 4
    - Price (REAL) # Example values: 672.64, 430.72, 121.99, 120.74, 645.05

Table: sqlite_sequence
  Columns:
    - name (TEXT) # Example values: transactions_1k
    - seq (TEXT) # Example values: 1000

Table: yearmonth
  Columns:
    - CustomerID (INTEGER) (Primary Key) # Example values: 39, 63, 172, 603, 1492
    - Date (TEXT) (Primary Key) # Example values: 201112, 201201, 201202, 201203, 201204
    - Consumption (REAL) # Example values: 528.3, 1598.28, 1931.36, 1497.14, 51.06


**Instructions:**
- Only use columns listed in the schema.
- Do not use any other columns or tables not mentioned in the schema.
- Ensure the SQL query is valid and executable.
- Use proper SQL syntax and conventions.
- Generate a complete SQL query that answers the question.
- Use the correct SQL dialect for the database, i.e., sqlite.
- Do not include any explanations or comments in the SQL output.
- Your output must start with ```sql and end with ```.

***Hints***
- Amount of more SMEs = Total of SMEs pay using Currency CZK - Total of SMEs pay using Currency EUR

Question: Is it true that more SMEs pay in Czech koruna than in euros? If so, how many more?...
```